In [ ]:
%matplotlib qt
import hyperspy.api as hs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
from scipy.ndimage import gaussian_filter
from hyperspy.misc.machine_learning import import_sklearn
import importlib
from alt_nonneg_least_squares import *
from implementations import *
from clustering_VCA import *
%load_ext autoreload
%autoreload 2

Loading Data From Hyperspy

In [ ]:
# Load the EDS data using hyperspy
s = hs.load("1 selection.bcf")[-1]
# cut out the "zero peak" of EDS data
for i in range(58):
    s.isig[i] = 0

# Take a patch of 100*100 to perform the decomposition.
s_temp = hs.signals.EDSTEMSpectrum(s.data[100:200,100:200,:])
s_temp.change_dtype('float')

# Perform the decomposition
s_temp.decomposition(algorithm="nmf",output_dimension=2)
loadings_hyperspy = s_temp.get_decomposition_loadings().data
factors_hyperspy = s_temp.get_decomposition_factors().data
plot( factors_hyperspy, "nmf_hyperspy.png" , "Decomposition using Hyperspy" )

Get the heuristic for Phase B and append.

In [ ]:
data = s_temp.data
bkg = get_Component_B( data, loadings_hyperspy )
data, loadings, factors, bkg = reshape( data, loadings_hyperspy, factors_hyperspy, bkg )
data, loadings, factors = append_Component_B( data, loadings, factors, bkg )

Run the Alternating Non negative Least Square Algorithm and save the plot

In [ ]:
# Intialize the W,H randomly
m = data.shape[0]
n = data.shape[1]
k = 2

W = np.zeros((m,k))
H = np.zeros((k,n))

for i in range(m):
	for j in range(k):
		W[i,j] = np.random.uniform(0,1)

for i in range(k):
	for j in range(n):
		H[i,j] = np.random.uniform(0,1)

# Intialize using ALS for better convergence. 
# Using the heuristic which is present in data[0,:] after the decomposition.
W,H = ALS( data, W, H, 10, data[0,:] )
# Decompose using ALternate Least Square with parameter
# The parameters are obtained after tuning using the below function.
best_lambda_si,best_lambda_bkg,best_gamma = tune_hyper_parameters(data)
W,H = ANLS( data, W, H,20,1,0.01,0.01,10 )
print_loss(data,W,H)
plot( H, "nmf_anls.png" , "Decomposition using ANLS NMF" )

In [ ]:
# Load the EDS data for hyperspy again for the experiment below
s = hs.load("1 selection.bcf")[-1]
for i in range(58):
    s.isig[i] = 0

# Take a patch of 100*100 to perform the decomposition.
s_temp = hs.signals.EDSTEMSpectrum(s.data[100:200,100:200,:])
s_temp.change_dtype('float')

data = s_temp.data
data = data.reshape(100*100,2048)

Vector Component Analysis

In [ ]:
# Decomposition using Vertex Component Analysis
# This saves the resultant decomposition in VCA.png in the current folder
VCA( data )

Clustering Algorithms

In [ ]:
# Decomposition using Kmeans Clustering
# This saves the resultant decomposition in kmeans.png in the current folder
K_means( data )

In [ ]:
# Decomposition using Fuzzy Clustering
# This saves the resultant decomposition in fuzzy.png in the current folder
fuzzy_clustering( data )